### Binomial and Trinomial Tree

In [2]:
%%file Binomial_Vanilla.m
%Vanilla Option Pricing (Binomial Tree)

function[price, f_tree] = Binomial_Vanilla(Style, C_P, Model, S, X, r, T, vol, q, N)
    dt = T/N;
    
    switch Model
        case 'CRR'
            u = exp(vol*sqrt(dt));
            d = 1/u;
            p = (exp((r-q)*dt) - d) / (u - d);
        case 'JR'
            u = exp((r-q-1/2*vol^2)*dt + vol*sqrt(dt));
            d = exp((r-q-1/2*vol^2)*dt - vol*sqrt(dt));
            p = 1/2;
    end
    
    %콜/풋 구분
    if C_P == 'C'
        z = 1;
    else
        z = -1;
    end
    
    %주가 이항트리 구성
    S_tree = zeros(N+1, N+1);
    
    for j = N+1:-1:1
        for i = 1:j
            S_tree(i, j) = S*u^(j-i)*d^(i-1);
        end
    end
    
    %만기시 옵션 Payoff 계산
    f_tree = zeros(N+1, N+1);
    f_tree(:, N+1) = max(0, z*(S_tree(:, end)-X));
    
    %트리의 역행을 통한 노드별 옵션가치 계산
    for j = N:-1:1
        for i = 1:j
            switch Style
                case 'EU'
                    f_tree(i, j) = exp(-r*dt)*(p*f_tree(i, j+1) + (1-p)*f_tree(i+1, j+1));
                case 'AM'
                    f_tree(i, j) = exp(-r*dt)*(p*f_tree(i, j+1) + (1-p)*f_tree(i+1, j+1));
                    f_tree(i, j) = max(f_tree(i, j), z*(S_tree(i, j) - X));
            end
        end
    end
    
    price = f_tree(1, 1);
    
        

Created file '/Users/changsoo/Desktop/논문/Matlab/Binomial_Vanilla.m'.


In [4]:
%%file Trinomial_Vanilla.m
%Vanilla Option Pricing(Trinomial Tree)

function [price, f_tree] = Trinomial_Vanilla(Style, C_P, S, X, r, T, vol, q, N)
    dt = T/N; 
    u  = exp(vol * sqrt(3*dt)); d = 1/u;
    pu =  sqrt(dt / (12 * vol^2)) * (r - q - vol^2/2) + 1/6;
    pd = -sqrt(dt / (12 * vol^2)) * (r - q - vol^2/2) + 1/6;
    pm = 2/3;
    
    %콜/풋 구분
    switch C_P
        case 'C'
            z = 1;
        case 'P'
            z = -1;
    end
    
    %주가 삼항트리 구성
    S_tree = zeros(2*N+1, N+1);
    g = 0; h = 2 * N + 2;
    
    for j = N+1:-1:1
        g = g+1;
        h = h-1;
        for i = g:h
            S_tree(i, j) = S*u^(N+1-i);
        end
    end
    
    %만기시 option payoff 계산
    f_tree = zeros(2*N+1, N+1);
    f_tree(:, N+1) = max(0, z*(S_tree(:, end) - X));
    
    %트리의 역행을 통한 노드별 옵션가치 계산
    g = 1; h = 2*N+1;
    
    for j = N:-1:1
        g = g+1;
        h = h-1;
        for i = g:h
            switch Style
                case 'EU'
                    f_tree(i, j) = exp(-r*dt) * (pu*f_tree(i-1, j+1) + pm*f_tree(i, j+1) + pd*f_tree(i+1, j+1));
                case 'AM'
                    f_tree(i, j) = exp(-r*dt) * (pu*f_tree(i-1, j+1) + pm*f_tree(i, j+1) + pd*f_tree(i+1, j+1));
                    f_tree(i, j) = max(f_tree(i, j), z*(S_tree(i, j)-X));
            end
        end
    end
    
    price = f_tree(N+1, 1);
    

Created file '/Users/changsoo/Desktop/논문/Matlab/Trinomial_Vanilla.m'.


In [5]:
%%file Trinomial_KnockOut.m
% Knock-Out Option Pricing(Trinomial Tree)

function [last_price, f_tree] = Trinomial_KnockOut(Style, C_P, S, X, r, T, vol, KO, barrier, q, N)
    dt = T/N;
    u  = exp(vol*sqrt(3*dt)); d = 1/u;
    pu =  sqrt(dt / (12*vol^2)) * (r-q-vol^2/2) + 1/6;
    pd = -sqrt(dt / (12*vol^2)) * (r-q-vol^2/2) + 1/6;
    pm = 2/3;
    
    %콜/풋 구분
    switch C_P
        case 'C'
            z = 1;
        case 'P'
            z = -1;
    end
    
    %주가 삼항트리 구성
    S_tree = zeros(2*N+1, N+1);
    g = 0; h = 2* N + 2;
    for j = N+1:-1:1
        g = g+1; h = h-1;
        for i = g:h
            S_tree(i, j) = S*u^(N+1-i);
        end
    end

    %외장애물가격(H1)과 내장애물가격(H2) 계산
    ST = S_tree(:, end);
    [a, b] = min(abs(ST - barrier));
    
    if ST(b) == barrier || ST(1) < barrier || ST(end) > barrier
        H1 = ST(b);
        H2 = ST(b);
    elseif ST(b) > barrier
        H1 = ST(b);
        H2 = ST(b+1);
    else
        H1 = ST(b-1);
        H2 = ST(b);
    end
    
    f_tree = zeros(2*N+1, N+1);
    price  = [];
    for H = [H1, H2];
        switch KO
            case 'D'
                for i = 1:2*N+1
                    if ST(i) > H
                        f_tree(i, end) = max(0, z*(ST(i) - X));
                    else
                        f_tree(i, end) = 0;
                    end
                end
                
            case 'U'
                for i = 1:2*N+1
                    if ST(i) < H
                        f_tree(i, end) = max(0, z*(ST(i) - X));
                    else
                        f_tree(i, end) = 0;
                    end
                end
        end
        
        g = 1; h = 2*N+1;
        for j = N:-1:1
            g = g+1; h = h-1;
            for i = g:h
                switch KO
                    case 'D'
                        switch Style
                            case 'EU'
                                if S_tree(i, j) > H
                                    f_tree(i, j) = exp(-r*dt) * (pu*f_tree(i-1, j+1) + pm*f_tree(i, j+1) + pd*f_tree(i+1, j+1));
                                else
                                    f_tree(i, j) = 0;
                                end
                            case 'AM'
                                if S_tree(i, j) > H
                                    f_tree(i, j) = exp(-r*dt) * (pu*f_tree(i-1, j+1) + pm*f_tree(i, j+1) + pd*f_tree(i+1, j+1));
                                    f_tree(i, j) = max(f_tree(i, j), z * (S_tree(i, j) - X));
                                else
                                    f_tree(i, j) = 0;
                                end
                        end
                    case 'U'
                        switch Style
                            case 'EU'
                                if S_tree(i, j) < H
                                    f_tree(i, j) = exp(-r*dt) * (pu*f_tree(i-1, j+1) + pm*f_tree(i, j+1) + pd*f_tree(i+1, j+1));
                                else
                                    f_tree(i, j) = 0;
                                end
        
                            case 'AM'
                                if S_tree(i, j) < H
                                    f_tree(i, j) = exp(-r*dt) * (pu*f_tree(i-1, j+1) + pm*f_tree(i, j+1) + pd*f_tree(i+1, j+1));
                                    f_tree(i, j) = max(f_tree(i, j), z*(S_tree(i, j) - X));
                                else
                                    f_tree(i, j) = 0;
                                end
                        end
                end
            end
        end
        
        temp_p = f_tree(N+1, 1);
        price  = [price; temp_p];
    end
    
    last_price = interp1([H1;H2], price, barrier, 'linear');
    
             
                        
        
        
        

Created file '/Users/changsoo/Desktop/논문/Matlab/Trinomial_KnockOut.m'.


In [1]:
%%file Trinomial_KnockIn.m
% Knock-In Option Pricing(Trinomial Tree)
function [last_price, f_tree2] = Trinomial_KnockIn(Style, C_P, S, X, r, T, vol, KI, barrier, q, N)
    dt = T/N;
    u  = exp(vol*sqrt(3*dt)); d = 1/u;
    pu =  sqrt(dt / (12*vol^2)) * (r - q - vol^2 / 2) + 1/6;
    pd = -sqrt(dt / (12*vol^2)) * (r - q - vol^2 / 2) + 1/6;
    pm = 2/3;
    
    %1단계 : Knock-In 베리어 hitting 가정시 각 노드별 주가 및 옵션가격 계산
    [S_tree, f_tree] = Trinomial_Tree(Style, C_P, S, X, r, T, vol, q, N);
    
    %풋, 콜 구분
    switch C_P
        case 'C'
            z = 1;
        case 'P'
            z = -1;
    end
    
    %외장애물가격(H1)과 내장애물가격(H2) 계산
    ST = S_tree(:, end);
    [a, b] = min(abs(ST-barrier));
    
    if ST(b) == barrier || ST(1) < barrier || ST(end) > barrier
        H1 = ST(b);
        H2 = ST(b);
    elseif ST(b) > barrier
        H1 = ST(b);
        H2 = ST(b+1);
    else
        H1 = ST(b-1);
        H2 = ST(b);
    end
    
    f_tree2 = zeros(2*N+1, N+1);
    price   = [];
    
    for H = [H1, H2]       %H1, H2에 해당하는 옵션가격 각각 계산
        %2grade : Barrier No hitting Payoff
        switch KI
            case 'D'
                for i=1:2*N+1
                    if ST(i) <= H
                        f_tree2(i, end) = max(0, z*(ST(i) - X));
                    else
                        f_tree2(i, end) = 0;
                    end
                end
            case 'U'
                for i=1:2*N+1
                    if ST(i) >= H
                        f_tree2(i, end) = max(0, z*(ST(i) - X));
                    else
                        f_tree2(i, end) = 0;
                    end
                end
        end
        
        g = 1; h = 2*h+1;
        for j = N:-1:1
            g = g+1; h=h-1;
            for i = g:h
                %3 grade : Knock-In OPtion per Node
                switch KI
                    case 'D'
                        switch Style
                            case 'EU'
                                f_tree2(i, j) = exp(-r*dt) * (pu*f_tree2(i-1, j+1) + pm*f_tree2(i, j+1) + pd*f_tree2(i+1, j+1));
                                
                                if S_tree(i, j) <= H
                                    f_tree2(i, j) = f_tree1(i, j);
                                end
                            case 'AM'
                                f_tree2(i, j) = exp(-r*dt) * (pu*f_tree2(i-1, j+1) + pm*f_tree2(i, j+1) + pd*f_tree2(i+1, j+1));
                                if S_tree(i, j) <= H
                                    f_tree2(i, j) = f_tree1(i, j);
                                end
                                f_tree2(i, j) = max(f_tree2(i, j), z*(S_tree(i, j) - X));
                        end
                    case 'U'
                        switch Style
                            case 'EU'
                                f_tree2(i, j) = exp(-r*dt) * (pu*f_tree2(i-1, j+1) + pm*f_tree2(i, j+1) + pd*f_tree2(i+1, j+1));
                                
                                if S_tree(i, j) >= H
                                    f_tree2(i, j) = f_tree1(i, j);
                                end
                            case 'AM'
                                f_tree2(i, j) = exp(-r*dt) * (pu*f_tree2(i-1, j+1) + pm*f_tree2(i, j+1) + pd*f_tree2(i+1, j+1));
                                if S_tree(i, j) >= H
                                    f_tree2(i, j) = f_tree1(i, j);
                                end
                                f_tree2(i, j) = max(f_tree2(i, j), z*(S_tree(i, j) - X));
                        end
                end
            end
        end
        
        temp_p = f_tree2(N+1, 1);
        price = [price;temp_p];
    end
    
    last_price = interp1([H1;H2], price, barrier, 'linear');
    
    
                                
                        
    
    


Created file '/Users/changsoo/Desktop/논문/Matlab/Trinomial_KnockIn.m'.


In [5]:
[price, f_tree] = Binomial_Vanilla('AM', 'P', 'CRR', 100, 100, 0.05, 0.5, 0.3, 0.01, 10)


price =

    7.4342


f_tree =

  1 ~ 7번 열

    7.4342    4.5015    2.3092    0.9015    0.2040    0.0000         0
         0   10.3822    6.7001    3.7181    1.5984    0.4075    0.0000
         0         0   14.0890    9.6935    5.8406    2.7884    0.8139
         0         0         0   18.5224   13.5662    8.8994    4.7622
         0         0         0         0   23.5343   18.2661   13.0505
         0         0         0         0         0   28.4955   23.5343
         0         0         0         0         0         0   33.1348
         0         0         0         0         0         0         0
         0         0         0         0         0         0         0
         0         0         0         0         0         0         0
         0         0         0         0         0         0         0

  8 ~ 11번 열

         0         0         0         0
         0         0         0         0
    0.0000         0         0         0
    1.6259    0.0000         0       

In [8]:
[price, f_tree] = Trinomial_Vanilla('AM', 'P', 100, 100, 0.05, 0.5, 0.3, 0.01, 10)


price =

    7.3534


f_tree =

  1 ~ 7번 열

         0         0         0         0         0         0         0
         0         0         0         0         0         0         0
         0         0         0         0         0         0         0
         0         0         0         0         0         0         0
         0         0         0         0         0         0         0
         0         0         0         0         0         0         0
         0         0         0         0    0.0062    0.0014         0
         0         0         0    0.1253    0.0705    0.0315    0.0086
         0         0    0.8423    0.6470    0.4624    0.2950    0.1539
         0    3.0406    2.6989    2.3389    1.9596    1.5603    1.1427
    7.3534    6.9980    6.6141    6.1953    5.7340    5.2190    4.6313
         0   13.1330   12.8721   12.5978   12.3025   11.9862   11.6567
         0         0   20.8011   20.7456   20.7354   20.7354   20.7354
         0         0         0  

In [9]:
Trinomial_KnockOut('EU', 'C', 100, 100, 0.05, 0.5, 0.3, 'U', 130, 0.01, 1000)


ans =

    2.8434



In [11]:
Trinomial_KnockOut('EU', 'P', 100, 100, 0.05, 0.5, 0.3, 'D', 70, 0.01, 1000)


ans =

    4.6394



In [16]:
Trinomial_KnockIn('EU', 'C', 100, 100, 0.05, 0.5, 0.3, 'U', 130, 0.01, 1000)


ans =

    6.4981



In [17]:
Trinomial_KnockIn('EU', 'P', 100, 100, 0.05, 0.5, 0.3, 'D', 70, 0.01, 1000)


ans =

    2.7319

